# Inference code for Captcha Hacker

### Import Module

In [3]:
import torch
import pandas as pd
from pathlib import Path
import cv2
import numpy as np
import torchvision.transforms as tf

### Mount from Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Set Seed

In [5]:
torch.manual_seed(120)
np.random.seed(120)

### Model Structure

In [6]:
import torch.nn as nn
from torchvision.models import resnet18


class ResCAPTCHA(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = resnet18(weights="DEFAULT")
        self.model.fc = nn.Linear(512, 128)
        self.drop1 = nn.Dropout(0.3)
        self.dense1 = nn.Linear(in_features=128, out_features=36)
        self.dense2 = nn.Linear(in_features=128, out_features=36)
        self.dense3 = nn.Linear(in_features=128, out_features=36)
        self.dense4 = nn.Linear(in_features=128, out_features=36)

    def forward(self, input):
        x = self.model(input)
        x = self.drop1(x)
        output = [self.dense1(x), self.dense2(x), self.dense3(x), self.dense4(x)]
        return output

In [7]:
%cd "/content/drive/MyDrive/ML_HW5"

/content/drive/MyDrive/ML_HW5


### Predict

In [8]:
def predict(cur_task, model_path, length, model_type=""):
   
    model = ResCAPTCHA()
  
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    with torch.no_grad():
        test_path = Path(".") / "test"
        for path in test_path.rglob("*.png"):
            if str(path).split("/")[1] == cur_task:
                img = cv2.imread(str(path))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                img = img / 255.0
                img = np.reshape(img, (img.shape[0], img.shape[1], 1))
                img = img.transpose((2, 0, 1))

                img = torch.tensor(img.astype(np.float32))
                img = torch.reshape(img, (1, img.shape[0], img.shape[1], img.shape[2]))
                
                candidate = [{} for _ in range(length)]
                for _ in range(5):
                  cur_img = transform(img)
                  cur_img = cur_img.to(device)
                  output = model(cur_img)
                 
                  for i in range(length):
                      index = alphabet[torch.argmax(output[i])]
                      if index in candidate[i].keys():
                        candidate[i][index] += 1
                      else:
                        candidate[i][index] = 1
                result = ""
                for i in range(length):
                  result += max(candidate[i], key=candidate[i].get)  
                ''' file path matters '''
                prediction["filename"].append(str(path)[5:])
                prediction["label"].append(result)

### Predict and Write CSV file

In [9]:
alphabet = list("0123456789abcdefghijklmnopqrstuvwxyz")
prediction = {"filename": [], "label": []}
transform = tf.Compose(
    [
        tf.RandomRotation([0, 15]),
        tf.Lambda(lambda x: x.repeat(1, 3, 1, 1)),
    ]
)

predict(cur_task="task1", model_path="/content/drive/MyDrive/109550003_HW5/model1_res18.pth", length=1)
predict(cur_task="task2", model_path="/content/drive/MyDrive/109550003_HW5/model2_res18.pth", length=2)
predict(cur_task="task3", model_path="/content/drive/MyDrive/109550003_HW5/model4_res18.pth", length=4)

%cd "/content"
df = pd.DataFrame(prediction)
df.to_csv("submission.csv", index=False)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/content
